In [1]:
import os
import requests
import pandas as pd
import json
import datetime
from dotenv import load_dotenv
from pathlib import Path

In [18]:
weather_data = pd.read_csv(
    Path('cleaned_data/df_weather_data_01122022.csv')
)

blmn_locs = pd.read_csv(
    Path('blmn_locs.csv')
)

load_dotenv()

# Connecting API key in .env to weather_api_key function
weather_api_key = os.getenv("WEATHER_API_KEY")
display(type(weather_api_key))

#Setting Request URL and creating list for responses from API
request_url = "https://api.weatherstack.com/historical"

str

In [19]:
check_df = weather_data[['request.query', 'date']]
check_df['request.query']= check_df['request.query'].apply(str)
check_df['key'] = check_df['request.query'] + check_df['date']
check_list = check_df['key'].to_list()


C:\Users\carra\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\carra\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
dates = list()
start_date = datetime.date(2021, 3, 1)
numdays = 306
for x in range (0, numdays):
    dates.append((start_date + datetime.timedelta(days = x)).strftime("%Y-%m-%d"))


In [22]:
blmn_selection = blmn_locs[blmn_locs['State'].isin([' FL', ' GA', ' AZ', ' NV', ' SC', 'FL', 'GA', 'AZ', 'NV', 'SC']) ]

blmn_selection_unique = blmn_selection['Zip Code'].unique()

zip_codes = blmn_selection_unique
zip_codes = [str(int) for int in zip_codes]
len(zip_codes)


202

In [23]:
check_key = []

for zip in zip_codes:
    for date in dates:
        x = zip + date
        check_key.append(x)

    

In [24]:
len(check_key)

61812

In [25]:
def non_match_elements(check_key, check_list):
    non_match = []
    for i in check_key:
        if i not in check_list:
            non_match.append(i)
    return non_match

In [26]:
non_match = non_match_elements(check_key, check_list)
len(non_match)

2022

In [27]:
unique_key = check_df['key'].unique()
len(unique_key)

59790

In [28]:
to_add_date = []
to_add_zip = []
len(non_match[0])

15

In [29]:
for i in non_match:
    x = i[-10:]
    to_add_date.append(x)

In [30]:
for i in non_match:
    x = len(i) - 10
    y = i[:x]
    to_add_zip.append(y)
    


In [31]:
responses = []


In [32]:

for i in non_match:
    x = len(i) - 10
    date_api = i[-10:]
    zip_api = i[:x]
    
    params = dict(
    access_key=weather_api_key,
    query= zip_api,
    historical_date = date_api,
    hourly=1, 
    interval=24)
    
    
    rr= requests.get(request_url, params=params)
    data = json.loads(rr.text)
    responses.append(data)

In [33]:
with open('api_responses/weather_data_missing_1122022.json', 'w') as json_file:
    json.dump(responses, json_file)

In [43]:
df_weather_data = pd.read_csv(
    Path('cleaned_data/df_weather_data_updated.csv')
)
df_weather_data_add = pd.read_csv(
    Path('cleaned_data/df_weather_data_01122022.csv')
)
    

In [44]:

df_weather_data_add.tail()

,Unnamed: 0,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,location.lon,...,dewpoint,windchill,windgust,feelslike,date,mintemp,maxtemp,avgtemp,totalsnow,sunhour
60391,60391,Zipcode,32819,en,m,Orlando,USA,Florida,28.460,-81.476,...,15,23,21,24,2021-03-16,19,29,24,0.0,11.6
60392,60392,Zipcode,32789,en,m,Winter Park,USA,Florida,28.600,-81.357,...,16,24,19,24,2021-03-16,19,29,25,0.0,11.6
60393,60393,Zipcode,34239,en,m,Sarasota,USA,Florida,27.310,-82.518,...,18,22,26,23,2021-03-16,19,24,22,0.0,11.6
60394,60394,Zipcode,30346,en,m,Atlanta,USA,Georgia,33.928,-84.344,...,12,14,14,14,2021-03-16,11,18,16,0.0,6.0
60395,60395,Zipcode,33131,en,m,Miami,USA,Florida,25.767,-80.191,...,18,23,33,25,2021-03-16,22,24,23,0.0,11.6


In [45]:
df_weather_data = pd.concat([df_weather_data, df_weather_data_add], ignore_index=True, axis=0)

In [46]:
df_weather_data.tail()

,Unnamed: 0,Unnamed: 0.1,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,...,dewpoint,windchill,windgust,feelslike,date,mintemp,maxtemp,avgtemp,totalsnow,sunhour
109599,60391,NaN,Zipcode,32819,en,m,Orlando,USA,Florida,28.460,...,15,23,21,24,2021-03-16,19,29,24,0.0,11.6
109600,60392,NaN,Zipcode,32789,en,m,Winter Park,USA,Florida,28.600,...,16,24,19,24,2021-03-16,19,29,25,0.0,11.6
109601,60393,NaN,Zipcode,34239,en,m,Sarasota,USA,Florida,27.310,...,18,22,26,23,2021-03-16,19,24,22,0.0,11.6
109602,60394,NaN,Zipcode,30346,en,m,Atlanta,USA,Georgia,33.928,...,12,14,14,14,2021-03-16,11,18,16,0.0,6.0
109603,60395,NaN,Zipcode,33131,en,m,Miami,USA,Florida,25.767,...,18,23,33,25,2021-03-16,22,24,23,0.0,11.6


In [47]:
#exporting cleaned data to csv
df_weather_data.to_csv('cleaned_data/df_weather_data_updated.csv')